In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/googleplaystore_user_reviews.csv


In [2]:
#googleplaystore_user_reviews = pd.read_csv("/kaggle/input/google-play-store-apps/googleplaystore_user_reviews.csv")
#googleplaystore_user_reviews.shape

In [3]:
import pandas as pd
googleplaystore_user_reviews = pd.read_csv("../input/googleplaystore_user_reviews.csv")

In [4]:
googleplaystore_user_reviews.shape

(64295, 5)

In [5]:
googleplaystore_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 5 columns):
App                       64295 non-null object
Translated_Review         37427 non-null object
Sentiment                 37432 non-null object
Sentiment_Polarity        37432 non-null float64
Sentiment_Subjectivity    37432 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.5+ MB


In [6]:
googleplaystore_user_reviews.dropna(axis = 0, inplace = True)

In [7]:
googleplaystore_user_reviews['Sentiment']#.iloc[:-5]


0        Positive
1        Positive
3        Positive
4        Positive
5        Positive
           ...   
64222    Positive
64223    Positive
64226    Negative
64227    Positive
64230    Negative
Name: Sentiment, Length: 37427, dtype: object

In [8]:
googleplaystore_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37427 entries, 0 to 64230
Data columns (total 5 columns):
App                       37427 non-null object
Translated_Review         37427 non-null object
Sentiment                 37427 non-null object
Sentiment_Polarity        37427 non-null float64
Sentiment_Subjectivity    37427 non-null float64
dtypes: float64(2), object(3)
memory usage: 1.7+ MB


In [9]:
x = googleplaystore_user_reviews['Translated_Review']
y = googleplaystore_user_reviews['Sentiment'] 

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state = 0)  

In [11]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(22456,) (14971,) (22456,) (14971,)


In [12]:

from sklearn.feature_extraction.text import CountVectorizer  
# Fit the CountVectorizer to the training data  
vect = CountVectorizer(stop_words='english', strip_accents='ascii', max_features=1000).fit(X_train)  
len(vect.get_feature_names())

1000

In [13]:
X_train.shape

(22456,)

In [14]:

# transform the documents in the training data to a document-term matrix  
X_train_vectorized = vect.transform(X_train)  
X_train_vectorized

<22456x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 186127 stored elements in Compressed Sparse Row format>

In [15]:
X_train_vectorized.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
x_test_transformed = vect.transform(X_test) 

In [17]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(X_train_vectorized,y_train)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,dt.predict(x_test_transformed)))

from sklearn.metrics import accuracy_score
print("accuracy_score : ",accuracy_score(y_test,dt.predict(x_test_transformed))) 

[[2534  194  593]
 [ 144 1652  290]
 [ 654  366 8544]]
accuracy_score :  0.850310600494289


In [18]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(random_state=121)
rf.fit(X_train_vectorized,y_train)
rf.predict(x_test_transformed)

pd.DataFrame([X_test.values,rf.predict(x_test_transformed)]).T

rf.predict(x_test_transformed)[1:20] # first 20 sentiment prediction

array(['Neutral', 'Negative', 'Positive', 'Positive', 'Positive',
       'Neutral', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Negative', 'Positive', 'Positive', 'Positive',
       'Neutral', 'Positive', 'Positive', 'Negative'], dtype=object)

In [19]:
#test=vect.transform(["the app is pathetic"])         #Neutral
#test=vect.transform(["the app is worse"])
#test=vect.transform(["this app is worthless"])        #Neutral
#test=vect.transform(["this app sucks"])    
#test=vect.transform(["this app has a ridiculus UI"])   #Neutral
#test=vect.transform(["time consuming"])                #Neutral
test=vect.transform(["time consuming app"])            #positive


rf.predict(test) 

array(['Positive'], dtype=object)